In [128]:
import openfermion
import openfermionpyscf
from openfermion import MolecularData
from openfermionpyscf import run_pyscf
from openfermion.ops import FermionOperator, QubitOperator
from openfermion.transforms import jordan_wigner, bravyi_kitaev
from openfermion.transforms import get_fermion_operator
from openfermion.circuits import (uccsd_singlet_get_packed_amplitudes,
                               uccsd_singlet_generator, uccsd_generator,
                               uccsd_convert_amplitude_format)
import numpy as np

singlet_bool = True # Set general UCCSD or singlet UCCSD.

bond_len = 0.772#1.45
atom_1 = 'He'
atom_2 = 'H'
basis = 'sto3g'
multiplicity = 1
charge = 1

coordinate_1 = (0.0, 0.0, 0.0)
coordinate_2 = (0.0, 0.0, bond_len)
geometry = [(atom_1, coordinate_1), (atom_2, coordinate_2)]

molecule_data = MolecularData(geometry, basis, multiplicity, charge, description='Test')
#molecule.load()

# Set calculation parameters.
run_scf = 1
run_mp2 = 1
run_cisd = 0
run_ccsd = 0
run_fci = 1
delete_input = True
delete_output = True

# Run pyscf.
molecule = run_pyscf(molecule_data,
                     run_scf=run_scf,
                     run_mp2=run_mp2,
                     run_cisd=run_cisd,
                     run_ccsd=run_ccsd,
                     run_fci=run_fci)

#molecule.load()
#print(molecule)

ham = get_fermion_operator(molecule.get_molecular_hamiltonian())
ham_q = jordan_wigner(ham)
print('Hamiltonian:', '\n', ham_q, '\n')

scf = True      # Hartree-Fock.
mp2 = True      # Moller-Plesset 2.
cisd = True     # Configuration interaction singles and doubles.
ccsd = True     # Coupled cluster singles and doubles.
fci = True      # Full configuration interaction.

calculated_molecule = run_pyscf(molecule_data, scf, mp2, cisd, ccsd, fci)

if ccsd:
    ccsd_single_amps = calculated_molecule.ccsd_single_amps
    ccsd_double_amps = calculated_molecule.ccsd_double_amps

num_electrons = calculated_molecule.n_electrons
num_qubits = 2*calculated_molecule.n_orbitals

if singlet_bool:
    # Get singlet UCCSD generator.
    packed_amps = uccsd_singlet_get_packed_amplitudes(ccsd_single_amps,  ccsd_double_amps, num_qubits, num_electrons)
    ucc_sing = uccsd_singlet_generator(packed_amps, num_qubits, num_electrons)
    #print(ucc_sing)

else:
    # Get general UCCSD operator.
    ucc_op = uccsd_generator(ccsd_single_amps, ccsd_double_amps)
    #print(ucc_op)
    
ucc_q = jordan_wigner(ucc_sing)
print('UCCSD ansatz:', '\n', ucc_q)

Hamiltonian: 
 (-1.5419759528969692+0j) [] +
(-0.03640948355953764+0j) [X0 X1 Y2 Y3] +
(0.03640948355953764+0j) [X0 Y1 Y2 X3] +
(0.05246557037552685+0j) [X0 Z1 X2] +
(-0.009234661837974558+0j) [X0 Z1 X2 Z3] +
(0.04322997146404521+0j) [X0 X2] +
(0.03640948355953764+0j) [Y0 X1 X2 Y3] +
(-0.03640948355953764+0j) [Y0 Y1 X2 X3] +
(0.05246557037552685+0j) [Y0 Z1 Y2] +
(-0.009234661837974558+0j) [Y0 Z1 Y2 Z3] +
(0.04322997146404521+0j) [Y0 Y2] +
(0.7589136134172096+0j) [Z0] +
(0.04322997146404521+0j) [Z0 X1 Z2 X3] +
(0.04322997146404521+0j) [Z0 Y1 Z2 Y3] +
(0.23578727153769516+0j) [Z0 Z1] +
(0.12887693985157755+0j) [Z0 Z2] +
(0.1652864234111152+0j) [Z0 Z3] +
(0.05246557037552685+0j) [X1 Z2 X3] +
(-0.009234661837974558+0j) [X1 X3] +
(0.05246557037552685+0j) [Y1 Z2 Y3] +
(-0.009234661837974558+0j) [Y1 Y3] +
(0.7589136134172096+0j) [Z1] +
(0.1652864234111152+0j) [Z1 Z2] +
(0.12887693985157755+0j) [Z1 Z3] +
(0.19140054473500764+0j) [Z2] +
(0.18815905542064587+0j) [Z2 Z3] +
(0.19140054473500764+0j

In [129]:
def QubitOperator_to_dict(op, num_qubits):
    
    assert(type(op) == QubitOperator)
    
    op_dict = {}
    
    term_dict = op.terms
    terms = list(term_dict.keys())

    for t in terms:
        
        letters = ['I' for i in range(num_qubits)]

        for i in t:
            letters[i[0]] = i[1]
        
        p_string = ''.join(letters)        
        op_dict[p_string] = term_dict[t]
         
    return op_dict

ham = QubitOperator_to_dict(ham_q, num_qubits)
anz_terms = list((QubitOperator_to_dict(ucc_q, num_qubits)).keys())

In [130]:
terms_noncon = c.greedy_dfs(ham, 1, criterion='weight')[-1]
ham_noncon = {t:ham[t] for t in terms_noncon}

In [131]:
c.contextualQ_ham(ham)

True

In [132]:
model = c.quasi_model(ham_noncon)
fn_form = c.energy_function_form(ham_noncon, model)
gs_noncon = c.find_gs_noncon(ham_noncon)
ep_state = gs_noncon[1]

In [133]:
size_R = fn_form[0] + fn_form[1]

In [134]:
ep_state

[[-1, -1, -1], [-0.07087650179757096, -0.9974850983813939]]

In [137]:
model

(['ZIZI', 'IZII', 'IIIZ'],
 ['YZYI', 'ZIII'],
 {'IIII': [[], [], 1],
  'ZIII': [[], ['ZIII'], 1],
  'IZII': [['IZII'], [], 1],
  'ZZII': [['IZII'], ['ZIII'], 1],
  'IIZI': [['ZIZI'], ['ZIII'], 1],
  'IIIZ': [['IIIZ'], [], 1],
  'IIZZ': [['ZIZI', 'IIIZ'], ['ZIII'], 1],
  'ZIIZ': [['IIIZ'], ['ZIII'], 1],
  'IZZI': [['ZIZI', 'IZII'], ['ZIII'], 1],
  'ZIZI': [['ZIZI'], [], 1],
  'IZIZ': [['IZII', 'IIIZ'], [], 1],
  'YZYI': [[], ['YZYI'], 1],
  'XZXI': [['ZIZI'], ['YZYI'], (-1+0j)],
  'YIYI': [['IZII'], ['YZYI'], 1],
  'XIXI': [['ZIZI', 'IZII'], ['YZYI'], (-1+0j)],
  'YZYZ': [['IIIZ'], ['YZYI'], 1],
  'XZXZ': [['ZIZI', 'IIIZ'], ['YZYI'], (-1+0j)]})

In [135]:
def ontic_prob(ep_state, ontic_state):
    
    if ep_state[0] != ontic_state[0]:
        return 0
    
    else:
        prod = 1
        for index, r in enumerate(ep_state[1]):
            f = 1/2 * abs(r + ontic_state[1][index])
            prod *= f
        
        return prod    

def epistemic_dist(ep_state):
    size_G = len(ep_state[0])
    size_Ci = len(ep_state[1])
    size_R = size_G + size_Ci
    
    ep_prob = {}
    
    ontic_states = list(itertools.product([1, -1], repeat=size_R))
    
    for o in ontic_states:
        o_state = [list(o[0:size_G]), list(o[size_G:size_R])]
        o_prob = ontic_prob(ep_state, o_state)
        
        if o_prob != 0:
            ep_prob[o] = o_prob
    
    return ep_prob

In [136]:
epistemic_dist(ep_state)

{(-1, -1, -1, 1, 1): 0.0005841635473785683,
 (-1, -1, -1, 1, -1): 0.46397758555383595,
 (-1, -1, -1, -1, 1): 0.000673287261924494,
 (-1, -1, -1, -1, -1): 0.534764963636861}